# Assignment 3 Part A - Recurrent Neural Networks

Welcome to the first part of the third assignment!

We'll be covering the implementation of RNNs. The main ideas will be relatively understandable, but the implementations may get a little confusing. So read everything carefully!

In [2]:
# Make sure to run this cell without modifying anything in it!
import numpy as np

# Import the code in `mytorch/nn.py`
#from mytorch import nn

# Import function that checks correctness of your answers.
#from tests import compare_to_answer

# These iPython functions make it so imported code is automatically reimported here if they are changed
%load_ext autoreload
%autoreload 2

In [3]:
#nn.py
import numpy as np

class Tanh:
    """[Given] The Tanh activation function, similar to torch.nn.Tanh."""
    def forward(self, x):
        """
        Forward pass for the Tanh activation function.

        Args:
            x (np.array): Input array.

        Returns:
            np.array: Output array after applying tanh.
        """
        return np.tanh(x)

    def backward(self, state):
        """
        Backward pass for the Tanh activation function.
        Calculates the derivative of tanh(x) with respect to x, which is 1 - tanh(x)^2.

        Args:
            state (np.array): The output of the forward pass (tanh(x)).

        Returns:
            np.array: The gradient with respect to the input of tanh.
        """
        return 1 - (state**2)


class RNNCell:
    """RNNCell, similar to torch.nn.RNNCell.

    Args:
        input_size (int): Integer representing # input features expected by layer
        hidden_size (int): Integer representing # features in the outputted hidden state
    """
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Initialize activation function
        self.activation = Tanh()

        # Randomly initialize weights and biases ("Kaiming Uniform" init)
        # bound = np.sqrt(1 / hidden_size)  # Common for uniform, but often 1/sqrt(fan_in) or 1/sqrt(fan_out)
        # For RNNs, a common initialization is often smaller, for tanh, typically 1/sqrt(fan_in) where fan_in = input_size + hidden_size
        # A more common practice for tanh with uniform is Xavier/Glorot initialization, which is sqrt(6 / (fan_in + fan_out))
        # Given your 'bound' was based on hidden_size, let's refine this to be more standard.

        # Xavier/Glorot uniform initialization (suitable for tanh)
        # For weight_ih, fan_in = input_size, fan_out = hidden_size
        bound_ih = np.sqrt(6 / (input_size + hidden_size))
        self.weight_ih = np.random.uniform(low=-bound_ih, high=bound_ih, size=(hidden_size, input_size))
        self.bias_ih = np.random.uniform(low=-bound_ih, high=bound_ih, size=(hidden_size,))

        # For weight_hh, fan_in = hidden_size, fan_out = hidden_size
        bound_hh = np.sqrt(6 / (hidden_size + hidden_size)) # or just bound_ih if you want to keep it simpler
        self.weight_hh = np.random.uniform(low=-bound_hh, high=bound_hh, size=(hidden_size, hidden_size))
        self.bias_hh = np.random.uniform(low=-bound_hh, high=bound_hh, size=(hidden_size,))


        # Gradients, initialized to zeros. These will accumulate during backprop.
        self.grad_weight_ih = np.zeros((hidden_size, input_size))
        self.grad_weight_hh = np.zeros((hidden_size, hidden_size))

        self.grad_bias_ih = np.zeros(hidden_size)
        self.grad_bias_hh = np.zeros(hidden_size)

        # Store intermediate values for backward pass
        self.x_t = None
        self.h_prev = None
        self.y_t = None # Output of the linear combination before activation

    def forward(self, x_t, h_prev):
        """RNNCell forward (single timestep)

        Args:
            x_t (np.array): (batch_size, input_size) current timestep's input
            h_prev (np.array): (batch_size, hidden_size) the previous timestep's outputted hidden state

        Returns:
            np.array: (batch_size, hidden_size) current timestep's outputted hidden state (h_t)
        """
        # Store for backward pass
        self.x_t = x_t
        self.h_prev = h_prev

        # Calculate y_t: linear combination of inputs and previous hidden state
        # (batch_size, input_size) @ (input_size, hidden_size) + (batch_size, hidden_size) @ (hidden_size, hidden_size)
        # Transpose weights for correct dot product if input is (batch_size, features)
        term_ih = np.dot(x_t, self.weight_ih.T) + self.bias_ih
        term_hh = np.dot(h_prev, self.weight_hh.T) + self.bias_hh

        # y_t is the sum before activation
        self.y_t = term_ih + term_hh

        # Calculate h_t: apply activation function to y_t
        h_t = self.activation.forward(self.y_t)

        return h_t

    def backward(self, grad, h_t, x_t, h_prev):
        """RNNCell backward (single timestep)

        Args:
            grad (np.array): (batch_size, hidden_size) the gradient w.r.t. the output of the current hidden layer (dL/dh_t)
            h_t (np.array): (batch_size, hidden_size) the output of the forward pass at this timestep (used for activation derivative)
            x_t (np.array): (batch_size, input_size) current timestep's input (used for input weight gradient)
            h_prev (np.array): (batch_size, hidden_size) the previous timestep's outputted hidden state (used for recurrent weight gradient)

        Returns:
            np.array, np.array: shaped (batch_size, input_size) and (batch_size, hidden_size)
                                respectively, representing dL/dx_t and dL/dh_prev
        """
        # Backprop through the activation function
        # dL/dy_t = dL/dh_t * dh_t/dy_t = grad * activation.backward(h_t)
        grad_through_activation = grad * self.activation.backward(h_t) # Element-wise multiplication

        # Accumulate the gradients for the weights and biases
        # Gradients are summed over the batch dimension

        # dL/dW_ih = (dL/dy_t).T @ x_t
        self.grad_weight_ih += np.dot(grad_through_activation.T, x_t)
        # dL/db_ih = sum(dL/dy_t, axis=0)
        self.grad_bias_ih += np.sum(grad_through_activation, axis=0)

        # dL/dW_hh = (dL/dy_t).T @ h_prev
        self.grad_weight_hh += np.dot(grad_through_activation.T, h_prev)
        # dL/db_hh = sum(dL/dy_t, axis=0)
        self.grad_bias_hh += np.sum(grad_through_activation, axis=0)

        # Calculate gradients for the input and the previous hidden state
        # dL/dx_t = (dL/dy_t) @ W_ih
        dx = np.dot(grad_through_activation, self.weight_ih)
        # dL/dh_prev = (dL/dy_t) @ W_hh
        dh = np.dot(grad_through_activation, self.weight_hh)

        return dx, dh


class RNN:
    """RNN layer, similar to torch.nn.RNN.

    Assume our version has `batch_first=True` and `bidirectional=False`.

    Args:
        input_size (int): Number of input features expected by layer
        hidden_size (int): Number of features in the outputted hidden state
        num_layers (int): Number of RNNCells to stack, s.t. each layer feeds its output into the next layer.
    """
    def __init__(self, input_size, hidden_size, num_layers=2):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Initialize first RNNCell, then add on more if num_layers > 1
        # The first layer takes input_size, subsequent layers take hidden_size as input
        self.layers = [RNNCell(input_size, hidden_size)]
        for l in range(num_layers - 1):
            self.layers.append(RNNCell(hidden_size, hidden_size))

    def forward(self, x, h_0=None):
        """Forward propagation over multiple RNNCells, multiple timesteps.

        Args:
            x (np.array): (batch_size, seq_len, input_size)
                          Input. A batch of sequences that each have `input_size` features at each timestep
            h_0 (np.array): (num_layers, batch_size, hidden_size)
                            Initial hidden state (useful if you have prior context to give to this layer).
                            If not given, creates a zero array.

        Returns:
            output (np.array): (batch_size, seq_len, hidden_size)
                               Output of the last cell for each timestep
            h_n (np.array): (num_layers, batch_size, hidden_size)
                            The hidden state at the last timestep for each layer in the batch
        """
        batch_size, seq_len, _ = x.shape

        # [Given] Initialize tensor to store every hidden state at each time step and cell of the rnn
        # hiddens[t+1, l, b, h] stores h_t for timestep t, layer l, batch b.
        # hiddens[0, :, :, :] stores h_0 for each layer.
        hiddens = np.zeros((seq_len + 1, self.num_layers, batch_size, self.hidden_size))
        if h_0 is not None: # If given, store the initial hidden state
            hiddens[0,:,:,:] = h_0

        # Store the inputs at each timestep for backprop (x_t for the first layer)
        # We need to store previous layer's output (which is current layer's input) for backprop.
        # This will be `x` for the first layer, and `hiddens[t+1, l-1, :, :]` for subsequent layers.
        # Let's create a structure to store inputs to each RNNCell at each timestep.
        # This will be `(seq_len, num_layers, batch_size, input_size_for_layer)`
        # The input size for the first layer is input_size, for others it's hidden_size.
        # To simplify storage, let's store (seq_len, num_layers, batch_size, max_dim)
        # A simpler approach is to store the actual x_t and h_prev within each RNNCell during its forward pass,
        # which you've already started doing by setting `self.x_t` and `self.h_prev` in `RNNCell.forward`.
        # However, for RNN class backward, you'll need the sequence of these for each layer and timestep.
        # So it's better to manage this explicitly here.

        # We'll store inputs to each layer and timestep for backward pass
        # This structure helps: inputs_to_cells[t][l] = input_array
        # Initialize a list of lists to hold cell inputs at each (timestep, layer)
        self.cell_inputs = [[None for _ in range(self.num_layers)] for _ in range(seq_len)]
        self.cell_h_prevs = [[None for _ in range(self.num_layers)] for _ in range(seq_len)]

        # Process input, timestep by timestep, layer by layer.
        for t in range(seq_len):
            # Input to the first layer at current timestep `t`
            current_input_to_layer = x[:, t, :] # (batch_size, input_size)

            for l in range(self.num_layers):
                # Get the previous hidden state for the current cell
                h_prev = hiddens[t, l, :, :] # (batch_size, hidden_size)

                # Store current input and h_prev for backward pass
                self.cell_inputs[t][l] = current_input_to_layer
                self.cell_h_prevs[t][l] = h_prev

                # Perform forward pass for the current RNNCell
                h_t = self.layers[l].forward(current_input_to_layer, h_prev)

                # Store the new hidden state
                hiddens[t+1, l, :, :] = h_t

                # The output of the current layer becomes the input for the next layer
                current_input_to_layer = h_t # (batch_size, hidden_size)

        # [Given] Save the original input and hidden vectors we used, for backprop later.
        self.x = x
        self.hiddens = hiddens

        # [Given] Return the output and final hidden states (we transpose to make the batch_size dim first)
        # output is the last layer's hidden states for all timesteps (excluding h_0)
        output = hiddens[1:, -1, :, :].transpose(1, 0, 2)
        # h_n is the hidden state at the last timestep for each layer
        h_n = hiddens[-1, :, :, :]

        return output, h_n

    def backward(self, grad):
        """Back Propagation Through Time (BPTT) through multiple RNNCells.

        Args:
            grad (np.array): (batch_size, hidden_size)
                             Gradient of the loss w.r.t. output of last RNN cell at the last timestep (dL/dh_{T, L})

        Returns:
            dx (np.array): (batch_size, seq_len, input_size)
                           Gradient of loss w.r.t. input
            dh_0 (np.array) : (num_layers, batch_size, hidden_size)
                              Gradient of loss w.r.t. the initial hidden state h_0
        """
        batch_size, seq_len, input_size = self.x.shape

        # [Given] Initialize gradients of the input and initial hidden state
        dx = np.zeros((batch_size, seq_len, input_size))
        dh_0 = np.zeros((self.num_layers, batch_size, self.hidden_size))

        # Initialize dh_current_layer_timestep: This will accumulate gradients from next layer and next timestep
        # This effectively holds dL/dh_{t,l} as we backpropagate.
        # It needs to be initialized with the gradient from the output layer (grad) for the last timestep/layer.

        # Initialize the gradient flowing backwards from the final output `grad`
        # This `grad_h` represents dL/dh_t for the current (t, l) being processed.
        # Initially, it's the gradient from the loss w.r.t. the final hidden state `h_n`.
        grad_h_current = np.zeros((self.num_layers, batch_size, self.hidden_size))
        grad_h_current[-1, :, :] = grad # This is dL/dh_{T, L} where T=last timestep, L=last layer

        # Iterate backwards through timesteps
        for t in reversed(range(seq_len)):
            # Initialize d_input_to_next_layer for this timestep.
            # This accumulates gradients that need to be passed down to the previous layer.
            # It starts with the gradient coming from the next layer (or 0 if this is the last layer being processed from top).
            # It will be updated by the RNNCell.backward's 'dx' output.
            grad_from_next_layer = np.zeros((batch_size, self.hidden_size)) # This is dL/dh_{t, l+1} essentially

            # Iterate backwards through layers
            for l in reversed(range(self.num_layers)):
                # Get the relevant hidden states and inputs for this cell's backward pass
                h_t = self.hiddens[t+1, l, :, :]       # Current cell's output (h_t)
                h_prev_t = self.hiddens[t, l, :, :]     # Current cell's previous hidden state (h_{t-1})
                x_t_for_cell = self.cell_inputs[t][l]   # Input to this cell at this timestep

                # The 'grad' passed to RNNCell.backward is the sum of gradients from:
                # 1. The output of the current cell (dL/dh_t from the downstream loss)
                # 2. The gradient from the next layer (dL/dh_{t, l+1})
                # 3. The gradient from the next timestep (dL/dh_{t+1, l}) - handled by grad_h_current

                # The total gradient for h_t is `grad_h_current[l, :, :]` (gradient flowing from next timestep)
                # PLUS `grad_from_next_layer` (gradient flowing from next layer).
                # Note: `grad_from_next_layer` accumulates the `dx` from the layer above.
                # If it's the last layer (`l == self.num_layers - 1`), then `grad_from_next_layer`
                # comes from the 'grad' provided to RNN.backward.
                # However, the structure of `grad` in RNN.backward is only `dL/dh_{T,L}`.
                # Let's adjust `grad_h_current` to hold the total gradient from `L` and `T` for each (l,t).

                # The gradient for h_t at this specific (t, l) comes from:
                # 1. The initial 'grad' passed to RNN.backward (only if t is last_t and l is last_l).
                # 2. The gradient from the *next layer* at the *current timestep* (passed as dx from layer l+1).
                # 3. The gradient from the *next timestep* in the *current layer* (passed as dh from cell at t+1).

                current_grad_for_cell = grad_h_current[l, :, :] + grad_from_next_layer

                # Perform backward pass for the current RNNCell
                # It returns dL/dx_t_cell (gradient w.r.t. input to this cell) and dL/dh_prev_cell (gradient w.r.t. h_prev)
                dx_cell, dh_prev_cell = self.layers[l].backward(
                    current_grad_for_cell, h_t, x_t_for_cell, h_prev_t
                )

                # Store the gradients:
                # dL/dh_prev_cell becomes the gradient for h_{t-1} in this layer, which is added to grad_h_current
                grad_h_current[l, :, :] = dh_prev_cell

                # dL/dx_cell becomes the gradient for the input to this cell.
                # If it's the first layer (l=0), this is dL/dx[:, t, :].
                # If it's not the first layer, this is dL/dh_{t, l-1}, which will be added to the grad_from_next_layer for the layer below.
                if l == 0:
                    dx[:, t, :] = dx_cell # Gradient for the original input `x`
                else:
                    grad_from_next_layer = dx_cell # This will be added to `current_grad_for_cell` for layer `l-1`

        # The final dh_0 is what's left in grad_h_current for the first timestep (t=0)
        # dh_0 should be `hiddens[0,:,:,:]` after backprop.
        # It's already accumulated in `grad_h_current` through the loop
        dh_0 = grad_h_current

        return dx, dh_0

In [4]:
#tests.py

import numpy as np

# ------------
# Test methods
# ------------

def test_rnncell_forward_1(RNNCell):
    layer = RNNCell(3, 4)
    layer.weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    layer.bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])
    layer.weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    layer.bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])
    x = np.array([[1., 2., 3.],
                  [4., 5., 6.]])
    h_prev = np.array([[1., 2., 3., 4.],
                    [3., 2., 1., 0.]])

    out = layer.forward(x, h_prev)
    return out

def test_rnncell_forward_2(RNNCell):
    layer = RNNCell(2, 5)
    layer.weight_ih = np.array([[-0.005, -5.,],
                                [0., 1.,],
                                [3., 4.,],
                                [6., 2.,],
                                [0., 0.]])
    layer.bias_ih = np.array([[-0.0002, -2., 1., 1., 0.]])
    layer.weight_hh = np.array([[-5., -4., -3., -2., -1.],
                                [-4., -3., -2., -1., 0.],
                                [0., 1., 2., 3., 0.],
                                [4., 5., 6., 7., 6.],
                                [7., 8., 9., -0.25, 0.25]])
    layer.bias_hh = np.array([[0.001, 2., -3., -4., 0.]])
    x = np.array([[0.0005, 2.],
                  [-0.152, -0.025],
                  [0., -1.]])
    h_prev = np.array([[0.02, 0.0005, 0.15, 0.45, 0.25],
                       [-0.001, 2., 1., 0., -0.25],
                       [1., -2., -5., -10., 0.]])

    out = layer.forward(x, h_prev)
    return out

def test_rnncell_forward_3(RNNCell):
    layer = RNNCell(5, 2)
    layer.weight_ih = np.array([[-0.005, -0.01, 0.05, 0.15, 0.222],
                                [0.151, 1.10, 0.015, -0.002, 0.051]])
    layer.bias_ih = np.array([[0.11, -0.15]])
    layer.weight_hh = np.array([[-0.152, 0.112],
                                [-0.002, -1.01]])
    layer.bias_hh = np.array([[0.001, 2.]])
    x = np.array([[-0.001, -2.015, -0.125, -0.001, 1.025],
                  [0.122, -0.128, -0.0002, -0.01, 0.15],
                  [-0.01, -0.002, 0.0152, 0.123, 0.231],
                  [-0.821, 0.999, 0.251, 0.331, 0.025]])
    h_prev = np.array([[0.02, 0.0005],
                       [-0.001, 2.],
                       [1., -2.],
                       [0.025, 0.152]])

    out = layer.forward(x, h_prev)
    return out

def test_rnncell_backward_1(RNNCell):
    layer = RNNCell(3, 4)
    layer.weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    layer.bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])
    layer.weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    layer.bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])
    h_prev = np.array([[1., 2., 3., 4.],
                    [3., 2., 1., 0.]])

    grad = np.array([[ 0.402888  ,  0.01693988,  0.04669028,  0.54219921],
                     [-0.88253697,  0.11543817, -0.70723666, -0.69393529]])
    h_prev_l = np.array([[-0.62162275, -0.96164911,  0.21242244],
                         [ 0.27069328, -0.83331687, -0.6866582 ]])
    h_prev_t = np.array([[ 0.33983293,  0.93178091,  0.69990522, -0.15981829],
                         [ 0.12239934, -0.56387259, -0.9556718 , -0.31575391]])

    dx, dh = layer.backward(grad, h_prev, h_prev_l, h_prev_t)
    return dx, dh, layer.grad_weight_ih, layer.grad_weight_hh, layer.grad_bias_ih, layer.grad_bias_hh


def test_rnncell_backward_2(RNNCell):
    layer = RNNCell(2, 5)
    layer.weight_ih = np.array([[-0.005, -5.,],
                                [0., 1.,],
                                [3., 4.,],
                                [6., 2.,],
                                [0., 0.]])
    layer.bias_ih = np.array([[-0.0002, -2., 1., 1., 0.]])
    layer.weight_hh = np.array([[-5., -4., -3., -2., -1.],
                                [-4., -3., -2., -1., 0.],
                                [0., 1., 2., 3., 0.],
                                [4., 5., 6., 7., 6.],
                                [7., 8., 9., -0.25, 0.25]])
    layer.bias_hh = np.array([[0.001, 2., -3., -4., 0.]])

    h_prev = np.array([[0.02, 0.0005, 0.15, 0.45, 0.25],
                       [-0.001, 2., 1., 0., -0.25],
                       [1., -2., -5., -10., 0.]])

    grad = np.array([[-0.9207447 , -0.0299716 ,  0.93634652, -0.75164411, -0.73439064],
       [-0.09640495, -0.87471139,  0.72295136,  0.8407652 ,  0.94054401],
       [ 0.08871341, -0.30203741,  0.57196294, -0.05680726, -0.59568463]])
    h_prev_l = np.array([[-0.66812673,  0.54131346],
       [-0.09802792,  0.56934838],
       [-0.0081655 ,  0.59356404]])
    h_prev_t = np.array([[ 0.72465154,  0.04605758,  0.44489962, -0.81581832, -0.46412398],
       [-0.11491536, -0.90658557, -0.06506369,  0.93885842, -0.88547036],
       [-0.86381171, -0.59843711, -0.63817132, -0.06270253,  0.22630636]])

    dx, dh = layer.backward(grad, h_prev, h_prev_l, h_prev_t)
    return dx, dh, layer.grad_weight_ih, layer.grad_weight_hh, layer.grad_bias_ih, layer.grad_bias_hh

def test_rnncell_backward_3(RNNCell):
    layer = RNNCell(5, 2)
    layer.weight_ih = np.array([[-0.005, -0.01, 0.05, 0.15, 0.222],
                                [0.151, 1.10, 0.015, -0.002, 0.051]])
    layer.bias_ih = np.array([[0.11, -0.15]])
    layer.weight_hh = np.array([[-0.152, 0.112],
                                [-0.002, -1.01]])
    layer.bias_hh = np.array([[0.001, 2.]])
    h_prev = np.array([[0.02, 0.0005],
                       [-0.001, 2.],
                       [1., -2.],
                       [0.025, 0.152]])

    grad = np.array([[-0.19521881, -0.82817359],
       [-0.52215925, -0.16664658],
       [ 0.82267516, -0.52842318],
       [-0.78720939, -0.11186485]])
    h_prev_l = np.array([[-0.10070971, -0.14045025, -0.31836873,  0.22785118, -0.40093625],
       [ 0.01313353, -0.05711794,  0.37709152, -0.80443835,  0.53103029],
       [ 0.84935059,  0.00150989, -0.76366634,  0.10226739, -0.83378106],
       [-0.71682395,  0.10407237,  0.84127296,  0.25157057, -0.28898927]])
    h_prev_t = np.array([[-0.28169733,  0.05609386],
       [ 0.66147383, -0.41036188],
       [ 0.51908581, -0.31614092],
       [-0.337796  , -0.46155793]])

    dx, dh = layer.backward(grad, h_prev, h_prev_l, h_prev_t)
    return dx, dh, layer.grad_weight_ih, layer.grad_weight_hh, layer.grad_bias_ih, layer.grad_bias_hh

def test_rnn_forward_1(RNN):
    rnn = RNN(3, 4, num_layers=2)

    rnn.layers[0].weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    rnn.layers[0].bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])

    rnn.layers[0].weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    rnn.layers[0].bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])

    rnn.layers[1].weight_ih = np.array([[-0.48600084,  0.56108125,  0.60350991, -0.12398511],
       [-0.61426125,  0.74533839,  0.28829775,  0.39298129],
       [-0.23660597,  0.60180463, -0.17925026, -0.91010067],
       [ 0.32842213,  0.88810569,  0.47370219, -0.02676785]])
    rnn.layers[1].bias_ih = np.array([[ 0.80336632,  0.5080941 , -0.02041526, -0.23457551]])
    rnn.layers[1].weight_hh = np.array([[-0.66123341,  0.89060331,  0.52273452,  0.51176128],
       [ 0.56754451, -0.00703578,  0.7770004 ,  0.27559471],
       [ 0.55518779, -0.73064323,  0.9099351 ,  0.57656225],
       [ 0.36056783, -0.05294811,  0.47201524, -0.4093564 ]])
    rnn.layers[1].bias_hh = np.array([[-0.03656752, -0.7702112 ,  0.43095679,  0.89191036]])

    x = np.array([[[ 0.54963061, -0.05740221, -0.18798255],
        [-0.05139327,  0.31849613, -0.71975814],
        [ 0.25741578,  0.5026123 ,  0.64851701],
        [ 0.15833562,  0.45382923,  0.17070994],
        [-0.02513259,  0.7292614 ,  0.84537154]],

       [[ 0.96812713, -0.77282445, -0.95171846],
        [ 0.38731339, -0.83533687,  0.99131245],
        [ 0.04599233, -0.95944108,  0.23168402],
        [ 0.16541892,  0.13714912, -0.95989072],
        [ 0.51472085,  0.37945332,  0.4281448 ]]])
    h_0 = np.array([[[-0.71008325, -0.07277072,  0.78528585, -0.35171659],
        [ 0.21250086,  0.90839637, -0.7255993 ,  0.53047081]],

       [[-0.86511414, -0.94116341, -0.36088932,  0.69915346],
        [-0.53193134,  0.29990252, -0.27214273, -0.72523786]]])

    out, hiddens = rnn.forward(x, h_0)
    return out, hiddens

def test_rnn_forward_2(RNN):
    rnn = RNN(2, 5, num_layers=1)

    rnn.layers[0].weight_ih = np.array([[-0.005, -5.,],
                                [0., 1.,],
                                [3., 4.,],
                                [6., 2.,],
                                [0., 0.]])
    rnn.layers[0].bias_ih = np.array([[-0.0002, -2., 1., 1., 0.]])
    rnn.layers[0].weight_hh = np.array([[-5., -4., -3., -2., -1.],
                                [-4., -3., -2., -1., 0.],
                                [0., 1., 2., 3., 0.],
                                [4., 5., 6., 7., 6.],
                                [7., 8., 9., -0.25, 0.25]])
    rnn.layers[0].bias_hh = np.array([[0.001, 2., -3., -4., 0.]])
    x = np.array([[[-0.37172187,  0.75289209],
            [-0.74297175, -0.02525347],
            [ 0.53391313, -0.67597502],
            [-0.7237738 , -0.26039564],
            [-0.80266149,  0.16000827]],

        [[ 0.21733297,  0.63915498],
            [-0.29721541, -0.99389827],
            [-0.41202637, -0.26365921],
            [-0.12466216, -0.60468756],
            [ 0.93159041, -0.78198452]],

        [[-0.87435205, -0.84892116],
            [ 0.46827638,  0.34440284],
            [ 0.88852153,  0.09396869],
            [-0.9570812 , -0.00129956],
            [ 0.24959137, -0.85093966]]])
    h_0 = np.array([[0.02, 0.0005, 0.15, 0.45, 0.25],
                       [-0.001, 2., 1., 0., -0.25],
                       [1., -2., -5., -10., 0.]])

    out, hiddens = rnn.forward(x, h_0)
    return out, hiddens

def test_rnn_forward_3(RNN):
    rnn = RNN(5, 2, num_layers=3)
    rnn.layers[0].weight_ih = np.array([[-0.005, -0.01, 0.05, 0.15, 0.222],
                                        [0.151, 1.10, 0.015, -0.002, 0.051]])
    rnn.layers[0].bias_ih = np.array([[0.11, -0.15]])
    rnn.layers[0].weight_hh = np.array([[-0.152, 0.112],
                                        [-0.002, -1.01]])
    rnn.layers[0].bias_hh = np.array([[0.001, 2.]])


    rnn.layers[1].weight_ih = np.array([[ 0.00372146, -0.82457555],
       [ 0.87680038, -0.46820705]])
    rnn.layers[1].bias_ih = np.array([ 0.6543994 , -0.49759433])
    rnn.layers[1].weight_hh = np.array([[-0.49185334, -0.22758022],
       [ 0.14179349,  0.74545256]])
    rnn.layers[1].bias_hh = np.array([ 0.10898453, -0.62921271])

    rnn.layers[2].weight_ih = np.array([[ 0.43795292, -0.39165178],
       [-0.43341205,  0.97362009]])
    rnn.layers[2].bias_ih = np.array([-0.30573836,  0.65735053])
    rnn.layers[2].weight_hh = np.array([[-0.03211809,  0.92450192],
       [-0.8617065 ,  0.03840494]])
    rnn.layers[2].bias_hh = np.array([-0.77441866,  0.05598034])


    x = np.array([[[-0.17857984, -0.93947819, -0.67213643, -0.69538802,
          0.20826295],
        [ 0.70933672,  0.55675034, -0.31722017,  0.96747612,
          0.69028081],
        [ 0.92119125, -0.13467066, -0.35187102, -0.20076277,
         -0.72281168]],

       [[ 0.36549078, -0.40412135, -0.50507736, -0.96217543,
         -0.68987774],
        [ 0.47556309, -0.21759229,  0.88103497, -0.77756477,
         -0.47552185],
        [-0.21578696,  0.55293293,  0.68532053,  0.78959926,
         -0.56555195]]])
    h_0 = np.array([[[-0.81155394, -0.67203904],
        [ 0.21009179, -0.28347711]],

       [[ 0.43134709,  0.19829147],
        [-0.20346234,  0.59861199]],

       [[-0.73887404, -0.843644  ],
        [-0.36426267,  0.3433386 ]]])

    out, hiddens = rnn.forward(x, h_0)
    return out, hiddens

def test_rnn_backward_1(RNN):
    rnn = RNN(3, 4, num_layers=2)

    rnn.layers[0].weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    rnn.layers[0].bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])

    rnn.layers[0].weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    rnn.layers[0].bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])

    rnn.layers[1].weight_ih = np.array([[-0.48600084,  0.56108125,  0.60350991, -0.12398511],
       [-0.61426125,  0.74533839,  0.28829775,  0.39298129],
       [-0.23660597,  0.60180463, -0.17925026, -0.91010067],
       [ 0.32842213,  0.88810569,  0.47370219, -0.02676785]])
    rnn.layers[1].bias_ih = np.array([[ 0.80336632,  0.5080941 , -0.02041526, -0.23457551]])
    rnn.layers[1].weight_hh = np.array([[-0.66123341,  0.89060331,  0.52273452,  0.51176128],
       [ 0.56754451, -0.00703578,  0.7770004 ,  0.27559471],
       [ 0.55518779, -0.73064323,  0.9099351 ,  0.57656225],
       [ 0.36056783, -0.05294811,  0.47201524, -0.4093564 ]])
    rnn.layers[1].bias_hh = np.array([[-0.03656752, -0.7702112 ,  0.43095679,  0.89191036]])

    x = np.array([[[ 0.54963061, -0.05740221, -0.18798255],
        [-0.05139327,  0.31849613, -0.71975814],
        [ 0.25741578,  0.5026123 ,  0.64851701],
        [ 0.15833562,  0.45382923,  0.17070994],
        [-0.02513259,  0.7292614 ,  0.84537154]],

       [[ 0.96812713, -0.77282445, -0.95171846],
        [ 0.38731339, -0.83533687,  0.99131245],
        [ 0.04599233, -0.95944108,  0.23168402],
        [ 0.16541892,  0.13714912, -0.95989072],
        [ 0.51472085,  0.37945332,  0.4281448 ]]])
    h_0 = np.array([[[-0.71008325, -0.07277072,  0.78528585, -0.35171659],
        [ 0.21250086,  0.90839637, -0.7255993 ,  0.53047081]],

       [[-0.86511414, -0.94116341, -0.36088932,  0.69915346],
        [-0.53193134,  0.29990252, -0.27214273, -0.72523786]]])

    rnn.forward(x, h_0)

    grad = np.array([[-0.1, 0.2, -0.3, 0.4],
                     [-0.4, 0.5, -0.6, 0.7]])

    dx, dh_0 = rnn.backward(grad)

    # Gather all the gradients in a single list
    layer_gradients = []
    for n in range(rnn.num_layers):
        layer_gradients.append([rnn.layers[n].grad_weight_ih, rnn.layers[n].grad_weight_hh, rnn.layers[n].grad_bias_ih, rnn.layers[n].grad_bias_hh])

    return dx, dh_0, layer_gradients

def test_rnn_backward_2(RNN):
    rnn = RNN(2, 5, num_layers=1)

    rnn.layers[0].weight_ih = np.array([[-0.005, -5.,],
                                [0., 1.,],
                                [3., 4.,],
                                [6., 2.,],
                                [0., 0.]])
    rnn.layers[0].bias_ih = np.array([[-0.0002, -2., 1., 1., 0.]])
    rnn.layers[0].weight_hh = np.array([[-5., -4., -3., -2., -1.],
                                [-4., -3., -2., -1., 0.],
                                [0., 1., 2., 3., 0.],
                                [4., 5., 6., 7., 6.],
                                [7., 8., 9., -0.25, 0.25]])
    rnn.layers[0].bias_hh = np.array([[0.001, 2., -3., -4., 0.]])
    x = np.array([[[-0.37172187,  0.75289209],
            [-0.74297175, -0.02525347],
            [ 0.53391313, -0.67597502],
            [-0.7237738 , -0.26039564],
            [-0.80266149,  0.16000827]],

        [[ 0.21733297,  0.63915498],
            [-0.29721541, -0.99389827],
            [-0.41202637, -0.26365921],
            [-0.12466216, -0.60468756],
            [ 0.93159041, -0.78198452]],

        [[-0.87435205, -0.84892116],
            [ 0.46827638,  0.34440284],
            [ 0.88852153,  0.09396869],
            [-0.9570812 , -0.00129956],
            [ 0.24959137, -0.85093966]]])
    h_0 = np.array([[0.02, 0.0005, 0.15, 0.45, 0.25],
                       [-0.001, 2., 1., 0., -0.25],
                       [1., -2., -5., -10., 0.]])

    rnn.forward(x, h_0)

    grad = np.array([[ 0.93211174, -0.38224902, -0.48032845,  0.74022526, -0.24474702],
       [ 0.0575357 ,  0.86414712,  0.30018032,  0.95602853,  0.09027887],
       [-0.62221448,  0.56082445, -0.75347751,  0.4888543 ,  0.80749852]])

    dx, dh_0 = rnn.backward(grad)

    # Gather all the gradients in a single list
    layer_gradients = []
    for n in range(rnn.num_layers):
        layer_gradients.append([rnn.layers[n].grad_weight_ih, rnn.layers[n].grad_weight_hh, rnn.layers[n].grad_bias_ih, rnn.layers[n].grad_bias_hh])

    return dx, dh_0, layer_gradients

def test_rnn_backward_3(RNN):
    rnn = RNN(5, 2, num_layers=3)
    rnn.layers[0].weight_ih = np.array([[-0.005, -0.01, 0.05, 0.15, 0.222],
                                        [0.151, 1.10, 0.015, -0.002, 0.051]])
    rnn.layers[0].bias_ih = np.array([[0.11, -0.15]])
    rnn.layers[0].weight_hh = np.array([[-0.152, 0.112],
                                        [-0.002, -1.01]])
    rnn.layers[0].bias_hh = np.array([[0.001, 2.]])


    rnn.layers[1].weight_ih = np.array([[ 0.00372146, -0.82457555],
       [ 0.87680038, -0.46820705]])
    rnn.layers[1].bias_ih = np.array([ 0.6543994 , -0.49759433])
    rnn.layers[1].weight_hh = np.array([[-0.49185334, -0.22758022],
       [ 0.14179349,  0.74545256]])
    rnn.layers[1].bias_hh = np.array([ 0.10898453, -0.62921271])

    rnn.layers[2].weight_ih = np.array([[ 0.43795292, -0.39165178],
       [-0.43341205,  0.97362009]])
    rnn.layers[2].bias_ih = np.array([-0.30573836,  0.65735053])
    rnn.layers[2].weight_hh = np.array([[-0.03211809,  0.92450192],
       [-0.8617065 ,  0.03840494]])
    rnn.layers[2].bias_hh = np.array([-0.77441866,  0.05598034])


    x = np.array([[[-0.17857984, -0.93947819, -0.67213643, -0.69538802,
          0.20826295],
        [ 0.70933672,  0.55675034, -0.31722017,  0.96747612,
          0.69028081],
        [ 0.92119125, -0.13467066, -0.35187102, -0.20076277,
         -0.72281168]],

       [[ 0.36549078, -0.40412135, -0.50507736, -0.96217543,
         -0.68987774],
        [ 0.47556309, -0.21759229,  0.88103497, -0.77756477,
         -0.47552185],
        [-0.21578696,  0.55293293,  0.68532053,  0.78959926,
         -0.56555195]]])
    h_0 = np.array([[[-0.81155394, -0.67203904],
        [ 0.21009179, -0.28347711]],

       [[ 0.43134709,  0.19829147],
        [-0.20346234,  0.59861199]],

       [[-0.73887404, -0.843644  ],
        [-0.36426267,  0.3433386 ]]])

    rnn.forward(x, h_0)

    grad = np.array([[ 0.65155603, -0.78930043],
       [ 0.11431596,  0.36581844]])

    dx, dh_0 = rnn.backward(grad)

    # Gather all the gradients in a single list
    layer_gradients = []
    for n in range(rnn.num_layers):
        layer_gradients.append([rnn.layers[n].grad_weight_ih, rnn.layers[n].grad_weight_hh, rnn.layers[n].grad_bias_ih, rnn.layers[n].grad_bias_hh])

    return dx, dh_0, layer_gradients

# ---------------------
# General methods below
# ---------------------

def compare_to_answer(user_output, answer, test_name=None):
    # Check that the object type of user's answer is correct
    if not check_types_same(user_output, answer, test_name):
        return False
    # Check that the shape of the user's answer matches the expected shape
    if not check_shapes_same(user_output, answer, test_name):
        return False
    # Check that the values of the user's answer matches the expected values
    if not check_values_same(user_output, answer, test_name):
        return False
    # If passed all the above tests, return True
    return True

def check_types_same(user_output, answer, test_name=None):
    try:
        assert isinstance(user_output, type(answer))
    except Exception as e:
        if test_name:
            print(f'Incorrect object type for {test_name}.')
        print("Your output's type:", type(user_output))
        print("Expected type:", type(answer))
        return False
    return True

def check_shapes_same(user_output, answer, test_name=None):
    try:
        assert user_output.shape == answer.shape
    except Exception as e:
        if test_name:
            print(f'Incorrect shape for {test_name}.')
        print('Your shape:', user_output.shape)
        print('Your values:\n', user_output)
        print('Expected shape:', answer.shape)
        return False
    return True

def check_values_same(user_output, answer, test_name=None):
    try:
        assert np.allclose(user_output, answer)
    except Exception as e:
        if test_name:
            print(f'Incorrect values for {test_name}.')
        print('Your values:\n', user_output)
        return False
    return True


# Section 1 - RNN Cell


## Question 1.1 - `RNNCell.forward()`

We'll begin by implementing a basic Elman RNN cell.

In `mytorch/nn.py`, implement `RNNCell.forward()` using the following description:

<div style="text-align:center">
    <img src="images/rnn_cell.png" width="250"/>
</div>

Above is the a visualization of the cell. You can see the previous timestep's outputted hidden state $h_{t-1}$ on the top left, and the current timestep's input $x_t$ on the bottom. The green box is where our computations will happen.

Calculate and `return` $h_t$ like so:

$$\begin{align*}
    y_t &= x_t W^{T}_{ih}+b_{ih}+h_{(t-1)} W^{T}_{hh}+b_{hh} \\
    h_t &= \text{Tanh}(y_t)
\end{align*}$$

$$\begin{align*}
    &\text{Where $W^{T}_{ih}$ is a transposed weight matrix for processing the input data $x_t$ and} \\
    &\text{$W^{T}_{hh}$ is a transposed weight matrix for processing the previous hidden state $h_{(t-1)}$.} \\
    &\text{Tanh refers to the Tanh activation function.}
\end{align*}$$

**Notes**:
- We've provided you an implementation of `Tanh` in `nn.py`.
- `Tanh` is also already initialized for you in `RNNCell.__init__()` as `self.activation`.
    - When you want to use it, call `self.activation.forward()` with the appropriate input.
- You can refer to the official [torch docs](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html) for this object if you want.

In [6]:
def test_rnncell_forward_1(RNNCell):
    layer = RNNCell(3, 4)
    layer.weight_ih = np.array([[-3., -2., -1.],
                                [0., 1., 2.],
                                [3., 4., 5.],
                                [6., 7., 8.]])
    layer.bias_ih = np.array([[-1., -2., 3., 4.]])
    layer.weight_hh = np.array([[-8., -7., -6., -5.],
                                [-4., -3., -2., -1.],
                                [0., 1., 2., 3.],
                                [4., 5., 6., 7.]])
    layer.bias_hh = np.array([[1., 2., -3., -4.]])
    x = np.array([[1., 2., 3.],
                [4., 5., 6.]])
    h_prev = np.array([[1., 2., 3., 4.],
                    [3., 2., 1., 0.]])

    out = layer.forward(x, h_prev)
    return out

test_rnncell_forward_1(RNNCell)

array([[-1.        , -1.        ,  1.        ,  1.        ],
       [-1.        , -0.99505475,  1.        ,  1.        ]])

In [10]:
#from tests import test_rnncell_forward_1, test_rnncell_forward_2, test_rnncell_forward_3

answer_1 = test_rnncell_forward_1(RNNCell)
answer_2 = test_rnncell_forward_2(RNNCell)
answer_3 = test_rnncell_forward_3(RNNCell)
print(answer_1)
print(answer_2)
print(answer_3)

[[-1.         -1.          1.          1.        ]
 [-1.         -0.99505475  1.          1.        ]]
[[-1.          0.82379071  0.99999955  0.99999655  0.89450007]
 [-1.         -0.99999978  0.89450007  1.          1.        ]
 [ 1.          1.         -1.         -1.         -1.        ]]
[[ 0.33577323 -0.30660517]
 [ 0.35190119 -0.27725774]
 [-0.19202418  0.9991407 ]
 [ 0.18397031  0.99056243]]


If you passed the tests above, assign the string "Question 1 passed" to ans1 in the code cell below.

In [11]:
### GRADED

ans1 = "Question 1 passed"



## Question 1.2 - `RNNCell.backward()`

Now for backprop. There are actually *seven* gradients we need to calculate.

**Gradient of the loss w.r.t. input of activation**

First, we need to calculate the gradient w.r.t. the input of the `Tanh` activation. We need this to calculate the other gradients.

$$\nabla_{y_t} \text{Loss} = \nabla_{y_t} h_{t} \odot \nabla_{h_{t}} \text{Loss}$$

$$\begin{align*}
    &\text{Where $\nabla_{y_t} h_{t}$ is the gradient of the layer's output $h_t$ w.r.t. the input of the activation function $y_t$,} \\
    &\text{and $\odot$ is the element-wise multiplication operator.}
\end{align*}$$

Notes:
- You can use `self.activation.backward()`, but will have to give it a `state`.
    - The `state` is the output of `RNNCell.forward()` at the current timestep.

**Gradients of the loss w.r.t. the weights and biases at this timestep:**

Now we need to calculate and store these four gradients:

$$\begin{align*}
    \nabla_{W_{ih}} \text{Loss} += \nabla_{y_t} \text{Loss}^T h_{(t, l-1)} \\
    \nabla_{W_{hh}} \text{Loss} += \nabla_{y_t} \text{Loss}^T h_{(t-1, l)} \\
    \nabla_{b_{ih}} \text{Loss} += \sum_{n=0}^{N-1}{\nabla_{y_t} \text{Loss}_n} \\
    \nabla_{b_{hh}} \text{Loss} += \sum_{n=0}^{N-1}{\nabla_{y_t} \text{Loss}_n}
\end{align*}$$

Notes:
- The first two formulas use matrix multiplications.
- The last two formulas are simply summing across the `batch_size` axis, yielding a size `(hidden_size,)` array.
- The last two formulas have identical right hand sides on purpose.
    - Think about why this is; it has to do the partial derivatives w.r.t. each of those bias terms.
- Notice the `+=` in the above equations.
    - Similar to `nn.Conv1d` in assignment 2a, we use the weights and biases multiple times across different timesteps, so we need to accumulate their total influence over time.
    - Backprop will pass through this layer for each time step it was used.
        - It'll go in reverse order of time (last timestep -> first timestep)
        - By the first timestep, it'll have finished accumulating the weight/bias gradients that represent the params' total influence on the output

**Gradients of the loss w.r.t. input and prev hidden state:**

Almost done! Calculate and `return` these two gradients:

$$\begin{align*}
\nabla_{x_t} \text{Loss} = \nabla_{y_t} \text{Loss} W_{ih}\\
\nabla_{h_{(t-1)}} \text{Loss} = \nabla_{y_t} \text{Loss} W_{hh}
\end{align*}$$

Notes:

- Notice that these use `=` instead of `+=`
    - These are not accumulated. We simply pass them backward for previous timesteps or for previous layers.

In [13]:
def test_rnncell_backward_1(RNNCell):
    layer = RNNCell(3, 4)
    layer.weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    layer.bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])
    layer.weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    layer.bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])
    x = np.array([[1., 2., 3.],
                  [4., 5., 6.]])
    h_prev = np.array([[1., 2., 3., 4.],
                    [3., 2., 1., 0.]])

    grad = np.array([[ 0.402888  ,  0.01693988,  0.04669028,  0.54219921],
                     [-0.88253697,  0.11543817, -0.70723666, -0.69393529]])
    h_prev_l = np.array([[-0.62162275, -0.96164911,  0.21242244],
                         [ 0.27069328, -0.83331687, -0.6866582 ]])
    h_prev_t = np.array([[ 0.33983293,  0.93178091,  0.69990522, -0.15981829],
                         [ 0.12239934, -0.56387259, -0.9556718 , -0.31575391]])

    dx, dh = layer.backward(grad, h_prev, h_prev_l, h_prev_t)
    return dx, dh, layer.grad_weight_ih, layer.grad_weight_hh, layer.grad_bias_ih, layer.grad_bias_hh

test_rnncell_backward_1(RNNCell)

(array([[-0.58475826, -0.67033156, -0.75590486],
        [-0.12264157, -0.20364703, -0.28465248]]),
 array([[-0.32328674, -0.40886004, -0.49443334, -0.58000664],
        [-0.57872849, -0.51852803, -0.45832757, -0.39812711]]),
 array([[ 1.91117462, -5.88346356, -4.84800998],
        [-0.06215437,  0.33746039,  0.22700447],
        [ 0.23218992,  0.35919733, -0.07934451],
        [ 4.86780684,  8.3993488 , -1.25113283]]),
 array([[ 0.86417554, -3.98110726, -6.74732556, -2.22931599],
        [-0.05965885,  0.14792449,  0.29539408,  0.11747207],
        [-0.12693516, -0.34804089, -0.26143017,  0.05969569],
        [-2.84879441, -7.18687201, -5.02914647,  1.51891304]]),
 array([ 7.06029576, -0.39713415, -0.37352224, -8.82692344]),
 array([ 7.06029576, -0.39713415, -0.37352224, -8.82692344]))

In [15]:
#from tests import test_rnncell_backward_1, test_rnncell_backward_2, test_rnncell_backward_3

answer_1 = test_rnncell_backward_1(RNNCell)
answer_2 = test_rnncell_backward_2(RNNCell)
answer_3 = test_rnncell_backward_3(RNNCell)
print(answer_1)
print(answer_2)
print(answer_3)

(array([[-0.58475826, -0.67033156, -0.75590486],
       [-0.12264157, -0.20364703, -0.28465248]]), array([[-0.32328674, -0.40886004, -0.49443334, -0.58000664],
       [-0.57872849, -0.51852803, -0.45832757, -0.39812711]]), array([[ 1.91117462, -5.88346356, -4.84800998],
       [-0.06215437,  0.33746039,  0.22700447],
       [ 0.23218992,  0.35919733, -0.07934451],
       [ 4.86780684,  8.3993488 , -1.25113283]]), array([[ 0.86417554, -3.98110726, -6.74732556, -2.22931599],
       [-0.05965885,  0.14792449,  0.29539408,  0.11747207],
       [-0.12693516, -0.34804089, -0.26143017,  0.05969569],
       [-2.84879441, -7.18687201, -5.02914647,  1.51891304]]), array([ 7.06029576, -0.39713415, -0.37352224, -8.82692344]), array([ 7.06029576, -0.39713415, -0.37352224, -8.82692344]))
(array([[ -0.84617901,   7.03415296],
       [  5.04507322,   4.78768884],
       [ -7.43781924, -42.75449253]]), array([[-2.49541491, -3.81841158, -5.14140825,  0.59263013, -2.84836347],
       [-0.47913155,  3.771

If you passed the tests above, assign the string "Question 2 passed" to ans2 in the code cell below.

In [16]:
### GRADED

ans2 = "Question 2 passed"



# Section 2 - `RNN` layer

The `RNN` object contains one or more `RNNCell`s and handles forward and backward prop over these cells.

- The number of cells is specified by the `num_layers` parameter.
- Documentation for the actual object is [here](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
    - Assume `bidirectional=False` for this assignment.
        - In practice, bidirectionality is usually crucial for RNN-based layers. You'll even use a bidirectional RNN in the second part of this assignment.
        - We choose not to have you implement it because the idea of it is simple once you understand it, but the implementation is confusing/time-consuming.
        - For more understanding of bidirectionality, refer to lecture (or Google)!
    - Assume `batch_first=True` for this assignment.

## Question 2.1 - `RNN.forward()`


<div style="text-align:center">
    <img src="images/rnn_forward.png" width="350"/>
</div>

Above is a visualization of `RNN.forward()`. We're given an input with `seq_len=3`, and we have `num_layers=2` `RNNCell`s. The first layer (first `RNNCell`) is on the bottom, and is used **3 times**, once for each step of the input.

Really, the main idea is to pass each timestep of the input through each cell of your RNN in order. That's it.

The hard part is indexing and carefully overwriting objects to make your code more efficient.

**Pseudocode**
- Declare a `hidden` array to store all hidden states in, and store the initial hidden state `h_0` in it (if given)
- for each timestep `t`:
    - declare `x_t` by getting a slice of `x` at time `t` along the appropriate axis
        - `x_t` should be shaped `(batch_size, input_size)`
    - for each layer `l`:
        - set `x_t` to the output of the `l`th layer's forward pass
            - provide as inputs `x_t` and a slice of the `hiddens` vector at time `t` and layer index `l`
                - the `hiddens` vector slice should be shaped `(batch_size, hidden_size)`
        - store `x_t` in the `hiddens` vector at the `t+1`th time and the `l`th layer


In [18]:
def test_rnn_forward_1(RNN):
    rnn = RNN(3, 4, num_layers=2)

    rnn.layers[0].weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    rnn.layers[0].bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])

    rnn.layers[0].weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    rnn.layers[0].bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])

    rnn.layers[1].weight_ih = np.array([[-0.48600084,  0.56108125,  0.60350991, -0.12398511],
       [-0.61426125,  0.74533839,  0.28829775,  0.39298129],
       [-0.23660597,  0.60180463, -0.17925026, -0.91010067],
       [ 0.32842213,  0.88810569,  0.47370219, -0.02676785]])
    rnn.layers[1].bias_ih = np.array([[ 0.80336632,  0.5080941 , -0.02041526, -0.23457551]])
    rnn.layers[1].weight_hh = np.array([[-0.66123341,  0.89060331,  0.52273452,  0.51176128],
       [ 0.56754451, -0.00703578,  0.7770004 ,  0.27559471],
       [ 0.55518779, -0.73064323,  0.9099351 ,  0.57656225],
       [ 0.36056783, -0.05294811,  0.47201524, -0.4093564 ]])
    rnn.layers[1].bias_hh = np.array([[-0.03656752, -0.7702112 ,  0.43095679,  0.89191036]])

    x = np.array([[[ 0.54963061, -0.05740221, -0.18798255],
        [-0.05139327,  0.31849613, -0.71975814],
        [ 0.25741578,  0.5026123 ,  0.64851701],
        [ 0.15833562,  0.45382923,  0.17070994],
        [-0.02513259,  0.7292614 ,  0.84537154]],

       [[ 0.96812713, -0.77282445, -0.95171846],
        [ 0.38731339, -0.83533687,  0.99131245],
        [ 0.04599233, -0.95944108,  0.23168402],
        [ 0.16541892,  0.13714912, -0.95989072],
        [ 0.51472085,  0.37945332,  0.4281448 ]]])
    h_0 = np.array([[[-0.71008325, -0.07277072,  0.78528585, -0.35171659],
        [ 0.21250086,  0.90839637, -0.7255993 ,  0.53047081]],

       [[-0.86511414, -0.94116341, -0.36088932,  0.69915346],
        [-0.53193134,  0.29990252, -0.27214273, -0.72523786]]])

    out, hiddens = rnn.forward(x, h_0)
    return out, hiddens

test_rnn_forward_1(RNN)

(array([[[ 0.58465959, -0.68351586,  0.59540493, -0.02945261],
         [ 0.03991988,  0.45049385,  0.94544681,  0.82507986],
         [ 0.9716002 ,  0.6782135 ,  0.87452011,  0.66842751],
         [ 0.91622999,  0.83858554,  0.91754133,  0.81214755],
         [ 0.95477401,  0.86818172,  0.90529103,  0.80123028]],
 
        [[ 0.67953154, -0.77425086, -0.64209047,  0.4879158 ],
         [-0.4051153 , -0.18908843,  0.77091333,  0.43887862],
         [ 0.89971484,  0.18574096,  0.86605359,  0.59347805],
         [ 0.78076771,  0.76497082,  0.96071053,  0.80285804],
         [ 0.95580484,  0.84303051,  0.9127279 ,  0.78835446]]]),
 array([[[-0.04496337,  0.03866571,  0.08922074,  0.13932018],
         [-0.02003295,  0.02673   ,  0.06193733,  0.09699118]],
 
        [[ 0.95477401,  0.86818172,  0.90529103,  0.80123028],
         [ 0.95580484,  0.84303051,  0.9127279 ,  0.78835446]]]))

In [20]:
#from tests import test_rnn_forward_1, test_rnn_forward_2, test_rnn_forward_3

answer_1 = test_rnn_forward_1(RNN)
answer_2 = test_rnn_forward_2(RNN)
answer_3 = test_rnn_forward_3(RNN)
print(answer_1)
print(answer_2)
print(answer_3)

(array([[[ 0.58465959, -0.68351586,  0.59540493, -0.02945261],
        [ 0.03991988,  0.45049385,  0.94544681,  0.82507986],
        [ 0.9716002 ,  0.6782135 ,  0.87452011,  0.66842751],
        [ 0.91622999,  0.83858554,  0.91754133,  0.81214755],
        [ 0.95477401,  0.86818172,  0.90529103,  0.80123028]],

       [[ 0.67953154, -0.77425086, -0.64209047,  0.4879158 ],
        [-0.4051153 , -0.18908843,  0.77091333,  0.43887862],
        [ 0.89971484,  0.18574096,  0.86605359,  0.59347805],
        [ 0.78076771,  0.76497082,  0.96071053,  0.80285804],
        [ 0.95580484,  0.84303051,  0.9127279 ,  0.78835446]]]), array([[[-0.04496337,  0.03866571,  0.08922074,  0.13932018],
        [-0.02003295,  0.02673   ,  0.06193733,  0.09699118]],

       [[ 0.95477401,  0.86818172,  0.90529103,  0.80123028],
        [ 0.95580484,  0.84303051,  0.9127279 ,  0.78835446]]]))
(array([[[-0.99996409, -0.0784464 ,  0.91327301,  0.95691316,
          0.89450007],
        [-0.10338642,  0.89094357,  

If you passed the tests above, assign the string "Question 3 passed" to ans3 in the code cell below.

In [21]:
### GRADED

ans3 =  "Question 3 passed"


## Question 2.2 - RNN.backward()

This is the famous "[backpropagation through time](https://en.wikipedia.org/wiki/Backpropagation_through_time)" algorithm that was independently derived by hand in the 80's and 90's by multiple researchers. Although [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) software like torch's `autograd` and modern computers make calculating arbitrary derivatives trivial at this point in time.

The challenge comes from all of the information being passed around and stored. Multiple timesteps, multiple layers, and hidden states for each of these layers and timesteps. You essentially need to retrace the forward diagram in reverse order, passing every necessary tensor at the right step.

The indexing and explanation for this is truly unbearable. We've tried to simplify this, but the algorithm is just inherently messy.

**Pseudocode**
- Initialize `dx` and `dh_0`, the gradients of the input and initial hidden state.
    - Assign the last layer index of `dh_0` to be `grad` (given)
- for each timestep `t` in reverse order from `seq_len` to `1` (inclusive on both sides, see notes):
    - for each layer `l` in reverse order from `num_layers-1` to `1` (inclusive on both sides):
        - get `dx_t_l` and `dh_0[l]` from the backward pass of the `l`th layer
            - provide `dh_0[l]`, `hiddens[t][l]`, `hiddens[t][l-1]`, and `hiddens[t-1][l]` as inputs
                - See the docstring of `RNNCell.backward()` for what these are
            - we're calculating the gradient of loss w.r.t. the current cell
        - add `dx_t_l` to `dh_0[l-1]` using a `+=` operation
            - because the hidden state from the previous layer impacted the current cell, we need to add its gradient (its impact) to the gradient of the loss w.r.t. previous layer's hidden state
    - get `dx_t` and `dh_0[0]` from the backward pass of the `0`th layer
        - provide `dh_0[0]`, `hiddens[t][0]`, `x[:,t-1,:]`, and `hiddens[t-1][0]` as inputs
    - set `dx[:,t-1,:]` equal to `dx_t`
        - after making it back through all the cells, we're finally back at the input

**Notes:**
- By "inclusive on both sides", we mean `t`$\in$`[seq_len, seq_len-1, ..., 1]`.
    - For `l` it'd be `l`$\in$`[num_layers-1, num_layers-2, ..., 1]`.
- The reason we end the inner loop at `1` is because the `0`th layer has to be given a slice from `x`.
    - Notice that we're giving this slice of `x` as `h_prev_l` in `RNNCell.backward()`. This is the "previous layer", which doesn't exist before the `0`th layer. So we just give it the original input at that timestep.
    - If you can find a way to neatly merge this into the `for` loop, hats off to you.
- Notice: `RNNCell.backward()` handles storing gradients for us

In [22]:
def test_rnn_backward_1(RNN):
    rnn = RNN(3, 4, num_layers=2)

    rnn.layers[0].weight_ih = np.array([[-0.01, -0.02, -0.03],
                                [0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06],
                                [0.07, 0.08, 0.09]])
    rnn.layers[0].bias_ih = np.array([[-0.01, -0.02, 0.03, 0.04]])

    rnn.layers[0].weight_hh = np.array([[-0.08, -0.07, -0.06, -0.05],
                                [-0.04, -0.03, -0.02, -0.01],
                                [0., 0.01, 0.02, 0.03],
                                [0.04, 0.05, 0.06, 0.07]])
    rnn.layers[0].bias_hh = np.array([[0.01, 0.02, -0.03, -0.04]])

    rnn.layers[1].weight_ih = np.array([[-0.48600084,  0.56108125,  0.60350991, -0.12398511],
       [-0.61426125,  0.74533839,  0.28829775,  0.39298129],
       [-0.23660597,  0.60180463, -0.17925026, -0.91010067],
       [ 0.32842213,  0.88810569,  0.47370219, -0.02676785]])
    rnn.layers[1].bias_ih = np.array([[ 0.80336632,  0.5080941 , -0.02041526, -0.23457551]])
    rnn.layers[1].weight_hh = np.array([[-0.66123341,  0.89060331,  0.52273452,  0.51176128],
       [ 0.56754451, -0.00703578,  0.7770004 ,  0.27559471],
       [ 0.55518779, -0.73064323,  0.9099351 ,  0.57656225],
       [ 0.36056783, -0.05294811,  0.47201524, -0.4093564 ]])
    rnn.layers[1].bias_hh = np.array([[-0.03656752, -0.7702112 ,  0.43095679,  0.89191036]])

    x = np.array([[[ 0.54963061, -0.05740221, -0.18798255],
        [-0.05139327,  0.31849613, -0.71975814],
        [ 0.25741578,  0.5026123 ,  0.64851701],
        [ 0.15833562,  0.45382923,  0.17070994],
        [-0.02513259,  0.7292614 ,  0.84537154]],

       [[ 0.96812713, -0.77282445, -0.95171846],
        [ 0.38731339, -0.83533687,  0.99131245],
        [ 0.04599233, -0.95944108,  0.23168402],
        [ 0.16541892,  0.13714912, -0.95989072],
        [ 0.51472085,  0.37945332,  0.4281448 ]]])
    h_0 = np.array([[[-0.71008325, -0.07277072,  0.78528585, -0.35171659],
        [ 0.21250086,  0.90839637, -0.7255993 ,  0.53047081]],

       [[-0.86511414, -0.94116341, -0.36088932,  0.69915346],
        [-0.53193134,  0.29990252, -0.27214273, -0.72523786]]])

    rnn.forward(x, h_0)

    grad = np.array([[-0.1, 0.2, -0.3, 0.4],
                     [-0.4, 0.5, -0.6, 0.7]])

    dx, dh_0 = rnn.backward(grad)

    # Gather all the gradients in a single list
    layer_gradients = []
    for n in range(rnn.num_layers):
        layer_gradients.append([rnn.layers[n].grad_weight_ih, rnn.layers[n].grad_weight_hh, rnn.layers[n].grad_bias_ih, rnn.layers[n].grad_bias_hh])

    return dx, dh_0, layer_gradients

# We check the input and initial hidden state gradients, as well as gradients on every RNNCell
test_rnn_backward_1(RNN)

(array([[[ 6.20279385e-05,  1.04250814e-04,  1.46473689e-04],
         [-2.01947807e-05,  2.86394852e-05,  7.74737512e-05],
         [ 4.30260692e-04,  6.04680222e-04,  7.79099753e-04],
         [-1.72465287e-04, -1.07503187e-04, -4.25410876e-05],
         [ 8.87477391e-03,  1.13015595e-02,  1.37283451e-02]],
 
        [[-4.99980869e-04, -3.52623822e-04, -2.05266775e-04],
         [-9.61793332e-04, -6.87768596e-04, -4.13743861e-04],
         [ 3.01276612e-03,  4.22621242e-03,  5.43965873e-03],
         [ 1.24645376e-03,  2.00870228e-03,  2.77095080e-03],
         [ 1.88604891e-02,  2.41814534e-02,  2.95024177e-02]]]),
 array([[[-2.65690159e-04, -2.02933049e-04, -1.40175939e-04,
          -7.74188289e-05],
         [-1.98018804e-03, -1.74489055e-03, -1.50959307e-03,
          -1.27429558e-03]],
 
        [[ 5.98615748e-03, -4.28515946e-03,  3.59952843e-03,
          -1.66637869e-03],
         [ 3.03995010e-02, -2.45643144e-02,  2.57475335e-02,
          -4.13379042e-03]]]),
 [[array([[0

In [24]:
#from tests import test_rnn_backward_1, test_rnn_backward_2, test_rnn_backward_3

answer_1 = test_rnn_backward_1(RNN)
answer_2 = test_rnn_backward_2(RNN)
answer_3 = test_rnn_backward_3(RNN)

print(answer_1)
print(answer_2)
print(answer_3)

(array([[[ 6.20279385e-05,  1.04250814e-04,  1.46473689e-04],
        [-2.01947807e-05,  2.86394852e-05,  7.74737512e-05],
        [ 4.30260692e-04,  6.04680222e-04,  7.79099753e-04],
        [-1.72465287e-04, -1.07503187e-04, -4.25410876e-05],
        [ 8.87477391e-03,  1.13015595e-02,  1.37283451e-02]],

       [[-4.99980869e-04, -3.52623822e-04, -2.05266775e-04],
        [-9.61793332e-04, -6.87768596e-04, -4.13743861e-04],
        [ 3.01276612e-03,  4.22621242e-03,  5.43965873e-03],
        [ 1.24645376e-03,  2.00870228e-03,  2.77095080e-03],
        [ 1.88604891e-02,  2.41814534e-02,  2.95024177e-02]]]), array([[[-2.65690159e-04, -2.02933049e-04, -1.40175939e-04,
         -7.74188289e-05],
        [-1.98018804e-03, -1.74489055e-03, -1.50959307e-03,
         -1.27429558e-03]],

       [[ 5.98615748e-03, -4.28515946e-03,  3.59952843e-03,
         -1.66637869e-03],
        [ 3.03995010e-02, -2.45643144e-02,  2.57475335e-02,
         -4.13379042e-03]]]), [[array([[0.0037086 , 0.0401436

If you passed the tests above, assign the string "Question 4 passed" to ans4 in the code cell below.

In [25]:
### GRADED

ans4 = "Question 4 passed"



# Epilogue - Part A's

Good work on completing all three 'Part A' portions of each assignemnt!

The goal of each 'Part A' was really to convince you that deep learning is just math at work. The perception that deep learning is a completely opaque black box is misguided at best and harmful at worst. We hope you'll agree that the math and code wasn't fancy - just matrix operations, derivatives, and simple loops.

If you consolidate all the code for `mytorch` you wrote throughout the course, you have a basic functioning deep learning library. If the course had more time, you'd have been able to add modules like `Dropout`, the `Adam` optimizer, different convolutions, `LSTM`, etc. We encourage you to try implementing those yourself in this same library. You can look up descriptions of how they work, and try to recreate them. To check if your implementation is correct, compare it to the actual torch's implementation. That's essentially how we developed this library.

The actual PyTorch differs in one big way though - while we manually implemented backpropagation and `backward()` methods for this course, PyTorch actually often automates this process using [autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html), a package for [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

For details on how `autograd` works, read the links provided. For the second link, note that `autograd` uses "reverse accumulation". The reason we use this instead of forward is explained in the "reverse accumulation" section of that link.

But we're not quite done yet - the second part of this assignment still remains. We'll see you there.